### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
Total_Schools=len(school_data_complete['school_name'].unique())  #Count of Total Unique School Names
Total_Students=school_data_complete['Student ID'].count() #Count Student IDs
Total_Budget=sum(school_data_complete['budget'].unique()) # Since Budget repeates for each student,we get unique values and sum.
Avg_Math_Score=school_data_complete['math_score'].mean() 
Avg_Reading_Score=school_data_complete['reading_score'].mean()
#Math Pass %= (Count of students with Math Score >=70/Total_Students)*100
MathPassPercent=(len(school_data_complete[school_data_complete['math_score']>=70])/Total_Students)*100
#Reading Pass %= (Count of students with Reading Score >=70/Total_Students)*100
ReadingPassPercent=(len(school_data_complete[school_data_complete['reading_score']>=70])/Total_Students)*100
#MOverall Pass %= (Count of students with Math Score >=70 or reading score >=70/Total_Students)*100
OverallPassPercent=(len(school_data_complete[(school_data_complete['reading_score']>=70)&(school_data_complete['math_score']>=70)])/Total_Students)*100

#Create a Dataframe
pd.DataFrame({'Total Schools':[Total_Schools],
              'Total Students':['{:,}'.format(Total_Students)],
              'Total Budget':['${:,.2f}'.format(Total_Budget)],
              'Average Math Score':['{:.6f}'.format(Avg_Math_Score)],
              'Average Reading Score':['{:.5f}'.format(Avg_Reading_Score)],
              '% Passing Math':['{:.6f}'.format(MathPassPercent)],
              '% Passing Reading':['{:.6f}'.format(ReadingPassPercent)],
              '% Overall Passing':['{:.6f}'.format(OverallPassPercent)]
             })

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
# Make a copy of Complete School data DataFrame
school_data_complete_mod=school_data_complete.copy()

#Add Additional columns to new DataFrame
#This column will hold 1 if student passed in Math else will be 0
school_data_complete_mod['math_pass']=np.where(school_data_complete_mod['math_score']>=70,1,0)
#This column will hold 1 if student passed in Reading else will be 0
school_data_complete_mod['reading_pass']=np.where(school_data_complete_mod['reading_score']>=70,1,0)
#This column will hold 1 if student passed both in Math & Readin else will be 0
school_data_complete_mod['overall_pass']=np.where((school_data_complete_mod['math_score']>=70) & (school_data_complete_mod['reading_score']>=70),1,0)

#Groupby School Name & Type and Create a Summary DataFrame.
SchoolSummaryDF=school_data_complete_mod.groupby(['school_name','type']).agg(
    Total_Students=pd.NamedAgg(column='Student ID',aggfunc='count'), #Count of Students
    Total_School_Budget=pd.NamedAgg(column='budget',aggfunc='max'), #Since Budget Repeates get max value
    Average_Math_Score=pd.NamedAgg(column='math_score',aggfunc='mean'),
    Average_Reading_Score=pd.NamedAgg(column='reading_score',aggfunc='mean'),
    Total_Math_Pass=pd.NamedAgg(column='math_pass',aggfunc='sum'), # No.of Students Passed in Math i.e. sum of math_pass column
    Total_Reading_Pass=pd.NamedAgg(column='reading_pass',aggfunc='sum'), # No.of Students Passed in Reading i.e. sum of reading_pass column
    Total_Overall_Pass=pd.NamedAgg(column='overall_pass',aggfunc='sum') # No.of Students Passed in Math & Reading i.e. sum of overall_pass column
).reset_index().set_index('school_name') #Set School name as Index

#Caculate percentage Columns and Per Student Budget
SchoolSummaryDF['Per_Student_Budget']=SchoolSummaryDF['Total_School_Budget']/SchoolSummaryDF['Total_Students']
SchoolSummaryDF['% Passing Math']=(SchoolSummaryDF['Total_Math_Pass']/SchoolSummaryDF['Total_Students'])*100
SchoolSummaryDF['% Passing Reading']=(SchoolSummaryDF['Total_Reading_Pass']/SchoolSummaryDF['Total_Students'])*100
SchoolSummaryDF['% Overall Passing']=(SchoolSummaryDF['Total_Overall_Pass']/SchoolSummaryDF['Total_Students'])*100

#Drop additional columns that are not needed for display
SchoolSummaryDF.drop(columns=['Total_Math_Pass','Total_Reading_Pass','Total_Overall_Pass'],inplace=True)
#Columns added during group by have '_',replace '_' with space
SchoolSummaryDF.columns=SchoolSummaryDF.columns.str.replace('_',' ')
# For display, turn off index name
SchoolSummaryDF.index.name=None
#Rename Type column to School Type
SchoolSummaryDF.rename(columns={'type':'School Type'},inplace=True)

#Format Column Values
SchoolSummaryDF['Total School Budget']=SchoolSummaryDF['Total School Budget'].map("${:,.2f}".format)
SchoolSummaryDF['Per Student Budget']=SchoolSummaryDF['Per Student Budget'].map("${:.2f}".format)
SchoolSummaryDF=SchoolSummaryDF[['School Type','Total Students','Total School Budget','Per Student Budget','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing']]

#Display DataFrame
SchoolSummaryDF


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [5]:
#Sort the Summary DF created above based on overall % percent in desceding order. Then display top 5 using head method
SchoolSummaryDF.sort_values('% Overall Passing',ascending=False).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [4]:
#Sort the Summary DF created above based on overall % percent in ascending order. Then display top 5 using head method
SchoolSummaryDF.sort_values('% Overall Passing').head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
#group the Original DataFrame by School name and Grade caculate Math Score Mean and store it in a DataFrame
MathScorebyGrade=school_data_complete.groupby(['school_name','grade'],as_index=False).agg({'math_score':'mean'})
#Pivot the dataframe. For display remove "grade" from display using rename_axis
MathScorebyGrade=MathScorebyGrade.pivot(index='school_name',columns='grade',values='math_score').rename_axis(None,axis=1)
#Trunf off Index Label for display
MathScorebyGrade.index.name=None
#Display datarframe with columns in asceding order of grades
MathScorebyGrade[['9th','10th','11th','12th']]

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
#group the Original DataFrame by School name and Grade caculate Reading Score Mean and store it in a DataFrame
ReadingScorebyGrade=school_data_complete.groupby(['school_name','grade'],as_index=False).agg({'reading_score':'mean'})
#Pivot the dataframe. For display remove "grade" from display using rename_axis
ReadingScorebyGrade=ReadingScorebyGrade.pivot(index='school_name',columns='grade',values='reading_score').rename_axis(None,axis=1)
#Trunf off Index Label for display
ReadingScorebyGrade.index.name=None
#Display datarframe with columns in asceding order of grades
ReadingScorebyGrade[['9th','10th','11th','12th']]

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [8]:
#In the School Summary DF created earlier,remove the currency formatting for Budget Columns and convert them to Numeric
SchoolSummaryDF['Total School Budget']=SchoolSummaryDF['Total School Budget'].str.replace('$','',regex=True).str.replace(',','',regex=True)
SchoolSummaryDF['Total School Budget']=pd.to_numeric(SchoolSummaryDF['Total School Budget'])
SchoolSummaryDF['Per Student Budget']=SchoolSummaryDF['Per Student Budget'].str.replace('$','',regex=True).str.replace(',','',regex=True)
SchoolSummaryDF['Per Student Budget']=pd.to_numeric(SchoolSummaryDF['Per Student Budget'])

#Set Speding Ranges and labels
SpendingRanges=['0','584','630','645','680']
SpendingRangeLabels=['<$585','$585-630','$630-645','$645-680']

#Calculate the Spending Range by Per Student Budget and it to the DF as a column
SchoolSummaryDF['Spending Ranges (Per Student)']=pd.cut(SchoolSummaryDF['Per Student Budget'],SpendingRanges,labels=SpendingRangeLabels)

# Group School Summary DF by Spending Range column, created above. Store it in a DF
ScoresbySchoolSpending=SchoolSummaryDF.groupby('Spending Ranges (Per Student)').agg(
Average_Math_Score=pd.NamedAgg(column='Average Math Score',aggfunc='mean'), #Avg. Math Score by Spending Range
Average_Reading_Score=pd.NamedAgg(column='Average Reading Score',aggfunc='mean'), #Avg. Reading Score by Spending Range
Pass_Math=pd.NamedAgg(column='% Passing Math',aggfunc='mean'), #Avg. Math Pass % by Spending Range
Pass_Reading=pd.NamedAgg(column='% Passing Reading',aggfunc='mean'), #Avg. Reading Pass % by Spending Range
Pass_Overall=pd.NamedAgg(column='% Overall Passing',aggfunc='mean') #Avg. Overall Pass % by Spending Range
)

#Replace '_' from column names with space for new calcualtion columns above
ScoresbySchoolSpending.columns=ScoresbySchoolSpending.columns.str.replace('_',' ')
#Rename Columns as needed for Display
ScoresbySchoolSpending.rename(columns={'Pass Math':'% Passing Math','Pass Reading':'% Passing Reading','Pass Overall':'% Overall Passing'},inplace=True)

#Format column values for display
ScoresbySchoolSpending['Average Math Score']=ScoresbySchoolSpending['Average Math Score'].map("{:.2f}".format)
ScoresbySchoolSpending['Average Reading Score']=ScoresbySchoolSpending['Average Reading Score'].map("{:.2f}".format)
ScoresbySchoolSpending['% Passing Math']=ScoresbySchoolSpending['% Passing Math'].map("{:.2f}".format)
ScoresbySchoolSpending['% Passing Reading']=ScoresbySchoolSpending['% Passing Reading'].map("{:.2f}".format)
ScoresbySchoolSpending['% Overall Passing']=ScoresbySchoolSpending['% Overall Passing'].map("{:.2f}".format)

#Display DF
ScoresbySchoolSpending


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
#Set School size Ranges and labels
SizeRanges=[0,999,2000,5000]
SizeRangeLabels=['Small (<1000)','Medium (1000-2000)','Large (2000-5000)']

#Calculate the School Size Range by Total Students and it to the DF as a column
SchoolSummaryDF['School Size']=pd.cut(SchoolSummaryDF['Total Students'],SizeRanges,labels=SizeRangeLabels)

# Group School Summary DF by School Size Range column, created above. Store it in a DF
ScoresbySchoolSize=SchoolSummaryDF.groupby('School Size').agg(
Average_Math_Score=pd.NamedAgg(column='Average Math Score',aggfunc='mean'), #Avg. Math Score by School Size Range
Average_Reading_Score=pd.NamedAgg(column='Average Reading Score',aggfunc='mean'), #Avg. Reading Score by School Size Range
Pass_Math=pd.NamedAgg(column='% Passing Math',aggfunc='mean'), #Avg. Math Pass % by School Size Range
Pass_Reading=pd.NamedAgg(column='% Passing Reading',aggfunc='mean'), #Avg. Reading Pass % by School Size Range
Pass_Overall=pd.NamedAgg(column='% Overall Passing',aggfunc='mean') #Avg. Overall Pass % by School Size Range
)

#Replace '_' from column names with space for new calcualtion columns above
ScoresbySchoolSize.columns=ScoresbySchoolSize.columns.str.replace('_',' ')
#Rename Columns as needed for Display
ScoresbySchoolSize.rename(columns={'Pass Math':'% Passing Math','Pass Reading':'% Passing Reading','Pass Overall':'% Overall Passing'},inplace=True)

#Display DF
ScoresbySchoolSize


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [10]:
# Group School Summary DF by School Type. Store it in a DF
ScoresbySchoolType=SchoolSummaryDF.groupby('School Type').agg(
Average_Math_Score=pd.NamedAgg(column='Average Math Score',aggfunc='mean'), #Avg. Math Score by School Type
Average_Reading_Score=pd.NamedAgg(column='Average Reading Score',aggfunc='mean'), #Avg. Reading Score by School Typee
Pass_Math=pd.NamedAgg(column='% Passing Math',aggfunc='mean'), #Avg. Math Pass % by School Type
Pass_Reading=pd.NamedAgg(column='% Passing Reading',aggfunc='mean'), #Avg. Reading Pass % by School Type
Pass_Overall=pd.NamedAgg(column='% Overall Passing',aggfunc='mean') #Avg. Overall Pass % by School Type
)

#Replace '_' from column names with space for new calcualtion columns above
ScoresbySchoolType.columns=ScoresbySchoolType.columns.str.replace('_',' ')
#Rename Columns as needed for Display
ScoresbySchoolType.rename(columns={'Pass Math':'% Passing Math','Pass Reading':'% Passing Reading','Pass Overall':'% Overall Passing'},inplace=True)

#Display DF
ScoresbySchoolType

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


# Trends

1) Charter Schools perform better than District schools. All top performing schools are Charter schools, all bottom performing schools are district schools.

2) Higher Spend per Student does not transform into better performance. As schools with lesser budget have performed better than the schools with high spend.

3) Small and Medium schools perform better than larger schools. Charter schools which perform better than district schools, have less students. So, looks like School size has impact on performance i.e. smaller and medium size schools perform better